# Question Answering

## Overview
Recall the overall workflow for retrieval augmented generation (RAG):

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

Let's load our vectorDB. 

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023. LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = '../docs/chroma'
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [6]:
print(vectordb._collection.count())

18


In [7]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

### RetrievalQA chain

In [10]:
from langchain.chains import RetrievalQA

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [12]:
result = qa_chain({"query": question})

/home/harry/anaconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [14]:
result["result"]

"I don't have enough information to determine the specific class or course you are referring to. If you can provide more context or details about the class, I may be able to assist you in identifying the major topics covered in that course."

# prompts

In [15]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [16]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [17]:
question = "Is probability a class topic?"

In [18]:
result = qa_chain({"query": question})

In [19]:
result["result"]

"I don't know, thanks for asking!"

In [20]:
result["source_documents"][0]

Document(page_content="Email\t(AIOU\temployees\tonly)\nFAQ's\t\nFinancial\tSupport\tScheme\t\nICMAP\tStudy\tMaterial\t\nJamia\tNama\t\nJobs\t\nMIT\tOpen\tCourseware\t\nNews\t&\tViews\t\nOAS\t(For\nProgram\tCoordinators)\t\nOverseas\t\nPakistan\tCitizen's\tPortal\t\nPakistan\tInfographic\t\nQuality\tEnhancement\t\nRTI\t(Right\tTo\tInformation)\nRegional\tCampuses\t\nSwift\tCenter\t\nTender\tNotices\t\nHome\t\nApply\tOnline\nÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\t\nÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\t\nÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\t\nÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\t\nÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ\tÂ

### RetrievalQA chain types
QA fails to preserve conversational history.

In [1]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

NameError: name 'RetrievalQA' is not defined

In [22]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

"Based on the provided context, there is no specific mention of probability as a class topic at AIOU (Allama Iqbal Open University). Therefore, I don't have information to confirm if probability is a class topic at AIOU."

In [23]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

"I don't have enough information to answer your question about why prerequisites are needed."